In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from concept_discovery.kmeans import kmeans_explore, kmeans_predict, plot_concepts
from imputer.load import load_imputer

from utils.utils import compute_significance, get_plot_drought, get_global, get_channel_wise

import subprocess
import json

d:\Anaconda\envs\deeplearning\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for efficiency.


In [2]:
import sys
print(sys.path)


['d:\\UCSD\\Fall 2024\\DSC 261\\Project\\CausalConceptTS', 'd:\\Anaconda\\envs\\deeplearning\\python39.zip', 'd:\\Anaconda\\envs\\deeplearning\\DLLs', 'd:\\Anaconda\\envs\\deeplearning\\lib', 'd:\\Anaconda\\envs\\deeplearning', '', 'd:\\Anaconda\\envs\\deeplearning\\lib\\site-packages', 'd:\\Anaconda\\envs\\deeplearning\\lib\\site-packages\\win32', 'd:\\Anaconda\\envs\\deeplearning\\lib\\site-packages\\win32\\lib', 'd:\\Anaconda\\envs\\deeplearning\\lib\\site-packages\\Pythonwin']


### Data loading

In [8]:
x_train = np.load('data/ptbxl/x_train.npy')
x_val = np.load('data/ptbxl/x_val.npy')
x_test = np.load('data/ptbxl/x_test.npy')

y_train = np.load('data/ptbxl/y_train.npy')
y_val = np.load('data/ptbxl/y_val.npy')
y_test = np.load('data/ptbxl/y_test.npy')

In [4]:
from sklearn.utils import shuffle

# Shuffle x and y together to maintain correspondence
x_train_, y_train_ = shuffle(x_train, y_train, random_state=42)
x_train_, y_train_ = x_train_[:1000], y_train_[:1000]

x_test_, y_test_ = shuffle(x_test, y_test, random_state=42)
x_test_, y_test_ = x_test_[:1000], y_test_[:1000]

x_val_, y_val_ = shuffle(x_val, y_val, random_state=42)
x_val_, y_val_ = x_val_[:1000], y_val_[:1000]

In [7]:
# Save as .npy files
np.save('data/ptbxl/x_train.npy', x_train_)
np.save('data/ptbxl/x_val.npy', x_val_)
np.save('data/ptbxl/x_test.npy', x_test_)

np.save('data/ptbxl/y_train.npy', y_train_)
np.save('data/ptbxl/y_val.npy', y_val_)
np.save('data/ptbxl/y_test.npy', y_test_)

NameError: name 'x_train_' is not defined

### Concepts loading or discovery

If you have already defined concepts, you should load here them as 
'train_concepts', 'val_concepts', and 'test_concets' and skip the next concept discovery step.
Each of these should be of the shape (N_concepts, N_samples, Lengh, Channels) and should be binary masks. e.g. 0's where the concept is present, and 1's where don't. (as per diffusion model requierement).

In [ ]:
# ks=number of clusters to investigate, it will return a plot to make a final decision

kmeans_explore(x_train,x_val,x_test,ks=10)

----:clustering at 1 clusters:----


AttributeError: 'NoneType' object has no attribute 'split'

In [15]:
x_train.shape

(9754, 248, 12)

In [9]:
# k=desired number of concepts

train_concepts,val_concepts,test_concepts = kmeans_predict(x_train,x_val,x_test,k=6)

np.save('data/drought/train_concepts.npy', train_concepts)
np.save('data/drought/val_concepts.npy', val_concepts)
np.save('data/drought/test_concepts.npy', test_concepts)

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
# plot your concepts

max_length = x_test.shape[1] # length of the ts for plotting (can be smaller than the total)
leadsn = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"] # name of your channels
index = np.where(y_test==1)[0][0] # index of a class sample
data = x_test # data to use (we used test for the index, so update accordly)
concepts = test_concepts # concepts of the desired split (test again)
save_fig = True 
savename = 'concepts.pdf'

plot_concepts(max_length, leadsn, index, data, concepts, savefig=save_fig, savename=savename)

### Classifier model loading or training

In this section, you can either load the model of your choice and assign it in a variable named 'classifier' or train an S4 model as in our main experiments paper

In [ ]:
# you data should be in 'data/'

# Construct the command to run the script
command = [
    'python', 'classifier/main.py',
    '--input-size', max_length,
    '--architecture', 's4',
    '--precision', 32,
    '--s4-n', 8,
    '--s4-h', 512,
    '--batch-size', 32,
    '--epochs', 20,
    '--input-channels', len(leadsn)
]

# it will save your model as first version (if there isn't any already)
subprocess.run(command, capture_output=True, text=True)

In [ ]:
# load the model configuration
with open('version_0/hparams.yaml') as f:
    hparams = yaml.load(f, Loader=SafeLoader)

namespace = Namespace(**hparams)

In [ ]:
# load the model and keep it in evaluation mode
model = Main(hparams=namespace).cuda()
model.load_weights_from_checkpoint('version_0/best_model.ckpt')
classifier = model.eval()

### Diffusion model training

In [ ]:
# create configuration files, one for each model

config_paths = []

for setting in ['class','norm']:

    config = {
        "diffusion_config": {
            "T": 200,
            "beta_0": 0.0001,
            "beta_T": 0.02
        },
        "wavenet_config": {
            "in_channels": len(leadsn), 
            "out_channels": len(leadsn),
            "num_res_layers": 36,
            "res_channels": 256, 
            "skip_channels": 256,
            "diffusion_step_embed_dim_in": 128,
            "diffusion_step_embed_dim_mid": 512,
            "diffusion_step_embed_dim_out": 512,
            "s4_lmax": x_train.shape[1],
            "s4_d_state": 64,
            "s4_dropout": 0.0,
            "s4_bidirectional": 1,
            "s4_layernorm": 1
        },
        "train_config": {
            "output_directory": f"/imputer/results/{setting}",
            "ckpt_iter": "max",
            "iters_per_logging": 100,
            "n_iters": 10000,
            "learning_rate": 2e-4,   
            "target_class": setting
        },
        "trainset_config": {
            "segment_length": x_train.shape[1]
        },
        "gen_config": {
            "output_directory": f"/imputer/results/{setting}",
            "ckpt_path": f"/imputer/results/{setting}/"
        }
    }

    # Save the configuration to a JSON file
    config_path = f'imputer/config_{setting}.json'
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)
    
    config_paths.append(config_path)

In [ ]:
config_paths

In [ ]:
# train each of the models

for config_path in config_paths:
    
    command = [
    'python', 'imputer/train.py',
    '--config', config_path]

    result = subprocess.run(command, capture_output=True, text=True)

### Diffusion model generation

In [ ]:
for config_path in config_paths:
    
    imputer = load_imputer(path_json=config_path,
                           ckpt_iter=10000)
    
    target_class = 'class'                                    
    target_generations = 40

    for mask_s, mask in zip(['A','B','C','D','E','F'], [test_concetps[0],
                                                             test_concepts[1],
                                                             test_concepts[2],
                                                             test_concepts[3],
                                                             test_concepts[4],
                                                             test_concepts[5]]):
        for g in range(1,target_generations+1):
            generation =  sampling(net,
                                  (len(x_test), x_test.shape[1], x_test.shape[2]),                                    
                                  diffusion_hyperparams, 
                                  cond = torch.from_numpy(samples).permute(0,2,1).cuda().float(),
                                  mask = torch.from_numpy(mask).permute(0,2,1).cuda().float())                                            

            outfile = f'{target_disease}_{mask_s}_{g}.npy'                  
            new_out = os.path.join(ckpt_path, outfile)
            np.save(new_out, generation.detach().cpu().numpy())
            print(outfile)

### Causal Concept Time Series Explainer (CausalConceptTS)

In [ ]:
global_sig_dou_s, global_sig_doc_s = get_global(concepts=['A','B','C','D','E','F'], 
                                                target_generations=target_generations, 
                                                l_channels=x_test.shape[2])

channel_sig_dou_s, channel_sig_doc_s = get_channel_wise(concepts=['A','B','C','D','E','F'],
                                                        target_generations=target_generations, 
                                                        l_channels=x_test.shape[2])

In [ ]:
get_plot_drought(global_sig_doc_g, 
         global_sig_doc_s, 
         global_do0_c, 
         channel_sig_doc_g, 
         channel_sig_doc_s, 
         channel_do0_c, 
         channels=leadsn,
        concepts=['A','B','C','D','E','F'],
         'do(causal).pdf')